# Exercice 4: connexion à l'API Mistral
L’objectif de cet exercice est de développer un mini-assistant conversationnel en Python qui utilise l’API Mistral pour répondre à des questions ou effectuer des tâches simples (résumé de texte, traduction, génération de code, etc.).

Avant de commencer à regarder et utiliser le code ci-dessous, il faut créer une clé api sur le site de Mistral. Sur [console.mistral.ai](https://console.mistral.ai), créer une clé API pour lechat. Il faudra créer un compte et souscrire à un plan expérimental (gratuit) pour l’obtenir. Copier cette clé dans la variable API_KEY.

In [1]:
!pip install mistralai
!pip install requests
!pip install python-dotenv

In [2]:
import requests
import os
import dotenv

dotenv.load_dotenv()
# Remplace ces valeurs par celles de ton compte Mistral
API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = os.getenv("MISTRAL_API_KEY")

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
}

data = {
    "model": "mistral-tiny",  # Remplace par le modèle que tu veux utiliser
    "messages": [
        {"role": "user", "content": "Bonjour, comment ça va ?"}
    ],
}

try:
    response = requests.post(API_URL, headers=headers, json=data)
    response.raise_for_status()  # Lève une erreur si la requête échoue
    print(response.json()["choices"][0]["message"]["content"])
except requests.exceptions.RequestException as e:
    print(f"Erreur lors de la requête : {e}")


Bonjour! Je suis parfaitement bien, et toi? Comment ça va? Je suis prêt à répondre à toutes vos questions en français. Si tu as besoin d'aide pour quelque chose, ne hésite pas à me demander.


Et voila, c'est tout, tu parles avec le chat! Maintenant, tansforme le code ci-dessous pour qu'il soit simple à utiliser. Créer une classe contenant une méthode pour initialiser la connexion à l'API et une pour envoyer un message au LLM et et récupérer son retour.

In [3]:
class ChatToMistral:
    def __init__(self, api_key):
        self.api_url = "https://api.mistral.ai/v1/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        }
    def send_message(self, message, model="mistral-tiny"):
        data = {
            "model": model,
            "messages": [
                {"role": "user", "content": message}
            ],
        }
        try:
            response = requests.post(self.api_url, headers=self.headers, json=data)
            response.raise_for_status()
            return response.json()["choices"][0]["message"]["content"]
        except requests.exceptions.RequestException as e:
            return f"Erreur lors de la requête : {e}"

L'objectif du reste de ce TP est maintenant de créer l'assistant en ajoutant des méthodes à cette classe.

Il faut pouvoir:

1. Résumer un texte long
2. Traduire un paragraphe en français vers l'anglais (ou d'autres langues en paramètres)
3. Générer du code Python pour une tâche donnée

Toutes les fonctions doivent être sur différents exemples. D'autres tâches aux choix peuvent être ajoutées à l'assistant sur le même principe.



In [21]:
class Assistant:
    def __init__(self):
        self.chat_client = ChatToMistral(API_KEY)

    def resumer(self, texte, longueur="concis en quelques phrases"):
        """
        Génère un prompt pour résumer un texte.

        Args:
            texte (str): Le texte à résumer.
            longueur (str): Une description de la longueur/style du résumé 
                           (ex: "concis en 3 phrases", "en une liste de points clés", "détaillé").
        """
        # On utilise des triples guillemets comme délimiteurs clairs
        question = f"""Résume le texte suivant de manière {longueur}.

                Texte à résumer :
                \"\"\"
                {texte}
                \"\"\"
                """
        return self._ask(question)
    
    def traduire(self, texte, langue_cible, langue_source="auto"):
        """
        Génère un prompt pour traduire un texte.

        Args:
            texte (str): Le texte à traduire.
            langue_cible (str): La langue de destination (ex: "Anglais", "Japonais").
            langue_source (str): Optionnel. La langue source pour aider le modèle.
                                 "auto" (défaut) laisse le modèle détecter.
        """
        
        source_instruction = f" depuis le {langue_source}" if langue_source != "auto" else ""

        # Des balises <texte> sont une autre excellente façon de délimiter
        question = f"""Traduis le texte suivant{source_instruction} en {langue_cible}. 
            Ne renvoie que la traduction en {langue_cible}, sans texte supplémentaire.

            <texte>
            {texte}
            </texte>
            """
        return self._ask(question)

    def generer_code(self, description, langage="Python"):
        """
        Génère un prompt pour écrire du code.

        Args:
            description (str): La description de ce que le code doit faire.
            langage (str): Le langage de programmation souhaité (ex: "Python", "JavaScript").
        """
        
        question = f"""Tu es un assistant expert en programmation {langage}.
        Génère un bloc de code fonctionnel en {langage} pour la description suivante.

        Description de la tâche :
        \"\"\"
        {description}
        \"\"\"

        Instructions de sortie :
        1.  Inclus des commentaires EN FRANCAIS clairs dans le code.
        2.  Utilise les meilleures pratiques pour le langage {langage} (ex: type hints en Python).
        3.  Réponds **uniquement** avec le bloc de code formaté en markdown (ex: ```{langage.lower()}) et rien d'autre. N'ajoute pas de salutations ou d'explications en dehors du bloc de code.
        """
        return self._ask(question)

    def _ask(self, question):
        return self.chat_client.send_message(question)

In [22]:
assistant = Assistant()
texte_a_resumer = """
3. Préparer un dataset contenant des séquences de 50 caractères (paramétrable) de
longueur en entrée et le caractère suivant cette séquence en target.
4. Transformer le dataset en séquence « one hot encodées »
5. Proposer et tester différentes architecture LSTM entrainées sur le dataset
précédemment généré.
"""

In [9]:
assistant.resumer(texte_a_resumer)

'Préparer un dataset avec séquences de caractères de longueur paramétrable, où chaque séquence est suivie d\'un target. Le dataset est ensuite converti en encodage "one hot". Ensuite, plusieurs architectures LSTM sont proposées et testées en utilisant le dataset généré.'

In [19]:
assistant.traduire("Je voudrais commander deux pizzas", "japonais", "français")

'ピザ二つを注文したい\n (Piza futatsu o teishoudai)'

In [24]:
print(assistant.generer_code("Un server qui affiche la température du jour en gros au centre de l'écran", "Python"))

```python
# Démarrage d'un serveur en Python pour afficher la température du jour en gros au centre de l'écran

from datetime import datetime
import requests
from tkinter import Tk, Label

# Fonction pour récupérer la température actuelle
def get_temperature():
    # URL de l'API météo pour récupérer la température actuelle
    url = "http://api.openweathermap.org/data/2.5/weather"

    # Paramètres de l'API : ville, clé d'API (récupérer votre propre clé sur OpenWeatherMap)
    params = {
        "q": "Paris,fr",
        "appid": "YOUR_API_KEY",
        "units": "metric"
    }

    # Requête à l'API pour récupérer les données
    response = requests.get(url, params=params)

    # Dictionnaire avec les données de météo
    data = response.json()

    # Température en degrés Celsius
    temperature = data["main"]["temp"]

    return temperature

# Fonction pour afficher la température dans la fenêtre graphique
def display_temperature(temperature):
    # Création de la fenêtre graphique
 